In [55]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [ ]:
import sqlalchemy

In [ ]:
import json
import re

In [3]:
from sodapy import Socrata

MyAppToken = '5YNrrmFobthjbfVSPQWEZWkkk'
username = 'jennifer.faith16@gmail.com'
password = 'Nashua1998!'

# Connect to dataset
client = Socrata("data.cms.gov",
                 MyAppToken,
                 username,
                 password)

results = client.get("s2uc-8wxp", limit = 1000000)

In [ ]:
#response = requests.get('https://data.cms.gov/resource/s2uc-8wxp.json?federal_provider_number==)

In [ ]:
response.json()

In [21]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [22]:
df.columns[0:20]

Index(['week_ending', 'federal_provider_number', 'provider_name',
       'provider_address', 'provider_city', 'provider_state',
       'provider_zip_code', 'submitted_data', 'county', 'reporting_interval',
       'geolocation', ':@computed_region_dz63_q6gx',
       ':@computed_region_ia25_mrsk', ':@computed_region_p3pw_njwh',
       ':@computed_region_x9pa_s2gq', ':@computed_region_vbss_8z7g',
       'passed_quality_assurance', 'residents_weekly_admissions',
       'residents_total_admissions', 'residents_weekly_confirmed'],
      dtype='object')

In [119]:
small = df[['federal_provider_number', 'provider_name', 'provider_address',
    'provider_city', 'provider_state', 'provider_zip_code', 'county', 'geolocation']]

In [120]:
small.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,geolocation
0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,NaN
1,045143,"CONCORDIA NURSING & REHAB, LLC",7 PROFESSIONAL DRIVE,BELLA VISTA,AR,72714,Benton,"{'type': 'Point', 'coordinates': [-94.25299200..."
2,045460,"COLONEL GLENN HEALTH AND REHAB, LLC",13700 DAVID O DODD ROAD,LITTLE ROCK,AR,72210,Pulaski,"{'type': 'Point', 'coordinates': [-92.410304, ..."
3,056317,GLENOAKS CONVALESCENT HOSPITAL,409 W. GLENOAKS BLVD.,GLENDALE,CA,91202,Los Angeles,"{'type': 'Point', 'coordinates': [-118.2619300..."
4,065309,WASHINGTON COUNTY NURSING HOME,676 W GREENHOUSE DR,AKRON,CO,80720,Washington,NaN


In [121]:
small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']

<ipython-input-121-7a40433a1853>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']


In [124]:
small['geolocation'].isnull().sum()

47717

In [125]:
no_dup = small.drop_duplicates(subset = ['federal_provider_number'])

In [126]:
no_dup['geolocation'].isnull().sum()

1199

In [127]:
# Fill location errors with 0s
no_dup['geolocation']=no_dup['geolocation'].fillna(0)

<ipython-input-127-5aa5a14d0a0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['geolocation']=no_dup['geolocation'].fillna(0)


In [130]:
def geo_column_edit(dictionary):
    if dictionary==0:
        pass
    else:
        values = dictionary.values()
        values_list = list(values)
        return (values_list[1][1], values_list[1][0])

In [131]:
no_dup['geolocation']=no_dup['geolocation'].apply(geo_column_edit)

<ipython-input-131-9e272aab63bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['geolocation']=no_dup['geolocation'].apply(geo_column_edit)


In [138]:
# NOTE: 1,199 have bad addresses. 
# These will not be mapped
no_dup[no_dup['geolocation'].isnull()].shape

(1199, 9)

In [139]:
no_dup.shape

(15407, 9)

In [180]:
new.to_csv('nhs_descriptors.csv')

## ALL TRIAL CODE GOES BELOW HERE

In [57]:
import geopy.geocoders
from geopy.geocoders import Nominatim

geopy.geocoders.options.default_user_agent = 'my_app/2'
geopy.geocoders.options.default_timeout = None

geolocator = Nominatim(user_agent="skilled_nursing")
location = geolocator.geocode("175 5th Avenue NYC")
print((location.latitude, location.longitude))

(40.741059199999995, -73.98964162240998)


In [58]:
location = geolocator.geocode("175 5th Avenue NYC")
print((location.latitude, location.longitude))

(40.741059199999995, -73.98964162240998)


In [82]:
a = small['full_address'][2]

In [146]:
location = geolocator.geocode('SEWARD, AK, 99664')
print((location.latitude, location.longitude))

(60.1219006, -149.438559)


In [75]:
print(location)

None


In [78]:
def geo_coordinates(address):
    location = geolocator.geocode(address)
    if location == None:
        return "Data Input Error"
    else:
        return (location.latitude, location.longitude)

In [158]:
geo_coordinates('86503')

(36.16763243080752, -109.68806973451998)

In [87]:
smaller = small.iloc[0:20]

In [89]:
smaller['geo']=smaller['full_address'].apply(geo_coordinates)

<ipython-input-89-f573131adb3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['geo']=smaller['full_address'].apply(geo_coordinates)


In [ ]:
small.to_csv('nhs_identifiers')

In [91]:
smaller['geolocation'][2]

{'type': 'Point', 'coordinates': [-92.410304, 34.708313]}

In [102]:
def geo_column_edit(dictionary):
    if dictionary==0:
        pass
    else:
        values = dictionary.values()
        values_list = list(values)
        return (values_list[1][1], values_list[1][0])

In [117]:
small['geolocation'] = small['geolocation'].fillna(0)

<ipython-input-117-348d1c545b56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['geolocation'] = small['geolocation'].fillna(0)


In [118]:
small['geolocation'] = small['geolocation'].apply(geo_column_edit)

AttributeError: 'tuple' object has no attribute 'values'

In [115]:
smaller

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,geolocation,full_address,geo
0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,None,"HIGHWAY 191 & HOSPITAL ROAD, CHINLE, AZ, 86503",Data Input Error
1,045143,"CONCORDIA NURSING & REHAB, LLC",7 PROFESSIONAL DRIVE,BELLA VISTA,AR,72714,Benton,"(36.474911, -94.25299200000002)","7 PROFESSIONAL DRIVE, BELLA VISTA, AR, 72714",Data Input Error
2,045460,"COLONEL GLENN HEALTH AND REHAB, LLC",13700 DAVID O DODD ROAD,LITTLE ROCK,AR,72210,Pulaski,"(34.708313, -92.410304)","13700 DAVID O DODD ROAD, LITTLE ROCK, AR, 72210","(34.708389, -92.410092)"
3,056317,GLENOAKS CONVALESCENT HOSPITAL,409 W. GLENOAKS BLVD.,GLENDALE,CA,91202,Los Angeles,"(34.159818, -118.26193000000002)","409 W. GLENOAKS BLVD., GLENDALE, CA, 91202","(34.159914072873924, -118.2624365591205)"
4,065309,WASHINGTON COUNTY NURSING HOME,676 W GREENHOUSE DR,AKRON,CO,80720,Washington,None,"676 W GREENHOUSE DR, AKRON, CO, 80720",Data Input Error
5,06A088,LITTLE SISTERS OF THE POOR - MULLEN HOME,3629 W 29TH AVE,DENVER,CO,80211,Denver,"(39.758508, -105.035758)","3629 W 29TH AVE, DENVER, CO, 80211","(39.759577, -105.03614551455956)"
6,105089,MANOR PINES CONVALESCENT CENTER,1701 NE 26TH ST,FORT LAUDERDALE,FL,33305,Broward,"(26.159670000000002, -80.12451)","1701 NE 26TH ST, FORT LAUDERDALE, FL, 33305","(26.1598026, -80.1180731)"
7,105384,CALUSA HARBOUR,2525 FIRST ST,FORT MYERS,FL,33901,Lee,"(26.647471, -81.864238)","2525 FIRST ST, FORT MYERS, FL, 33901","(26.64866775, -81.86512731343228)"
8,115275,"A.G. RHODES HOME, INC, THE","350 BOULVARD, S.E.",ATLANTA,GA,30312,Fulton,None,"350 BOULVARD, S.E., ATLANTA, GA, 30312",Data Input Error
9,145323,CARRIER MILLS NSG & REHAB CTR,6789 US RT 45,CARRIER MILLS,IL,62917,Saline,None,"6789 US RT 45, CARRIER MILLS, IL, 62917",Data Input Error


In [148]:
nulls = no_dup[no_dup['geolocation'].isnull()]

In [163]:
nulls['geolocation']

0         86503
4         80720
8         30312
9         62917
11        60471
          ...  
607659    77845
607683    76549
607696    75090
607859    78070
608000    77640
Name: geolocation, Length: 1199, dtype: object

In [159]:
nulls['geolocation']= nulls['provider_zip_code']

<ipython-input-159-b879417a5aeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nulls['geolocation']= nulls['provider_zip_code']


In [164]:
def geo_null(address):
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)

In [165]:
nulls['geolocation'] = nulls['geolocation'].apply(geo_null)

<ipython-input-165-d81eea947679>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nulls['geolocation'] = nulls['geolocation'].apply(geo_null)


In [174]:
nulls_z = nulls[['federal_provider_number', 'geolocation']]

In [175]:
new = no_dup.merge(nulls_z, on = 'federal_provider_number')

In [177]:
new.drop(columns = ['geolocation_x'], axis=1, inplace=True)

In [179]:
new.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,full_address,geolocation_y
0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,"HIGHWAY 191 & HOSPITAL ROAD, CHINLE, AZ, 86503","(36.16763243080752, -109.68806973451998)"
1,065309,WASHINGTON COUNTY NURSING HOME,676 W GREENHOUSE DR,AKRON,CO,80720,Washington,"676 W GREENHOUSE DR, AKRON, CO, 80720","(49.82296369811355, 24.514349484476256)"
2,115275,"A.G. RHODES HOME, INC, THE","350 BOULVARD, S.E.",ATLANTA,GA,30312,Fulton,"350 BOULVARD, S.E., ATLANTA, GA, 30312","(33.74310470677914, -84.37445028551396)"
3,145323,CARRIER MILLS NSG & REHAB CTR,6789 US RT 45,CARRIER MILLS,IL,62917,Saline,"6789 US RT 45, CARRIER MILLS, IL, 62917","(37.682745708232396, -88.62537025784138)"
4,145424,LANDMARK OF RICHTON PARK REHAB & NSG CTR,22660 SOUTH CICERO AVENUE,RICHTON PARK,IL,60471,Cook,"22660 SOUTH CICERO AVENUE, RICHTON PARK, IL, 6...","(41.48200539573889, -87.72284133679652)"
